## Load data and generate network graph

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import networkx as nx
import algo.net_helper as nh
import numpy as np
import matplotlib.pyplot as plt
import math

# PLEASE NOTE: 
# This script generates output files (graphs and assessed network files) in subdirectories
# of the current workingdir (from where you execute this script) -> "plots" and "net_out"

# network data input (as generated by NetAScore - see https://github.com/plus-mobilitylab/netascore)
base_dir = "/Users/christian/Documents/work/code/netascore/data" # set to your input data directory
compare_case_id = None
case_id = "denver_subset"    # the case_id refers to the geopackage file name
aoi_buffer_size = 5             # 5 for real-world net, 50 for generated grid (or half the cell size)
is_generated_net = False        # set to True if using a generated grid network (with reference case)
compare = False                 # set to True if using a generated grid network (with reference case)

debug_routes = False

edge_key = "edge_id" # CAUTION: currently, only "edge_id" is fully supported - using e.g. "osm_id" will cause errors when trying to join area back to edges
if is_generated_net and compare:
    case_id_prefix = str(case_id)
    case_id = case_id_prefix + "_subdiv"
    compare_case_id = case_id_prefix + "_simple"

case_str = f"{'gen_' if is_generated_net else ''}{case_id}"
net_file = f"{base_dir}/netascore_{case_str}.gpkg"
compare_net_file = None

cmap_diverging = "PiYG"
cmap_centr = "OrRd"
 
normalize = True

edges = gpd.read_file(net_file, layer="edge")
# if input does not have 'edge_id' column, generate it from index
if not 'edge_id' in edges.columns:
    edges['edge_id'] = edges.index
    print("created 'edge_id' column from gdf index.")
print(f"Loaded {len(edges)} network edges")
nodes = gpd.read_file(net_file, layer="node")
if "node_id" in nodes.columns:
    nodes.set_index('node_id', inplace=True)
    print("set index to node_id")
else:
    print("WARNING: 'node_id' is not present in nodes gdf - cannot set index.")
print("Nodes:", len(nodes))
# add edges ft+tf
net_routing = nh.netascore_to_routable_net(edges)
# generate NetworkX graph and find largest connected component
g = nx.from_pandas_edgelist(net_routing, source='from_node', target='to_node', edge_attr=True, create_using=nx.MultiDiGraph, edge_key=edge_key)
g = g.subgraph(max(nx.weakly_connected_components(g), key=len))
# filter edge list accordingly
filtered_edges = gpd.GeoDataFrame(nx.to_pandas_edgelist(g), crs=edges.crs)
e_ids = filtered_edges.edge_id.unique()
print("Edges in largest connected component:", len(e_ids))
edges = gpd.GeoDataFrame(edges.loc[edges.edge_id.isin(e_ids)])
# filter nodes accordingly to match edge subset for visualisation
filtered_nids = np.unique(np.append(edges.from_node.unique(), (edges.to_node.unique())))
nodes = gpd.GeoDataFrame(nodes[nodes.index.isin(filtered_nids)])

if compare_case_id is not None and compare_case_id != "":
    compare_case_str = f"{'gen_' if is_generated_net else ''}{compare_case_id}"
    compare_net_file = f"{base_dir}/netascore_{compare_case_str}.gpkg"
    compare_edges = gpd.read_file(compare_net_file, layer="edge")
    compare_nodes = gpd.read_file(compare_net_file, layer="node")
    compare_net_routing = nh.netascore_to_routable_net(compare_edges)
    compare_g = nx.from_pandas_edgelist(compare_net_routing, source='from_node', target='to_node', edge_attr=True, create_using=nx.MultiDiGraph, edge_key=edge_key)
    compare_g = compare_g.subgraph(max(nx.weakly_connected_components(compare_g), key=len))
    
# generate polygon representing covered AOI extent
aoi = nodes.unary_union.convex_hull.envelope
# for generated network: extend to match common ebtw centrality reference case conditions
if is_generated_net:
    aoi = aoi.buffer(aoi_buffer_size, resolution=1).envelope
else:
    aoi = aoi.buffer(aoi_buffer_size)

orig_crs = edges.crs
print(f"input CRS: {edges.crs}")

edges = edges.clip(aoi)

net_routing.head()

os.makedirs(os.path.join("plots", "svg"), exist_ok=True)
os.makedirs(os.path.join("net_out"), exist_ok=True)

In [ ]:
import pandas as pd
import shapely as sly
if debug_routes:
    # debugging of network connectivity etc.
    o_id = nodes.sample().index.values[0]
    d_id = nodes.sample().index.values[0]
    paths = [p for p in nx.all_shortest_paths(g, o_id, d_id, weight='length')]
    node_routes = []
    for p in paths:
        # direct line between nodes for simple vis
        node_routes.append(sly.geometry.LineString([[a.x, a.y] for a in nodes.loc[p].geometry.values]))
    map = nodes.set_crs(crs=32633, allow_override=True).explore()
    rdf = gpd.GeoDataFrame(node_routes, columns=['geometry'])
    rdf.set_crs(crs=32633, allow_override=True, inplace=True)
    rdf['rid'] = range(1,len(paths)+1)
    display(rdf.explore(m=map, column="rid"))


## compute standard centrality

In [ ]:
c_sp = nx.edge_betweenness_centrality(g, normalized=normalize, weight='length')
nh.add_centr_to_netascore(edges, c_sp, "sp", edge_key)

In [ ]:
if compare_net_file is not None:
    compare_c_sp = nx.edge_betweenness_centrality(compare_g, normalized=normalize, weight='length')
    nh.add_centr_to_netascore(compare_edges, compare_c_sp, "sp", edge_key)

## compute o/d node weights
### first: tesselate planar space based on network edges and compute each polygon's area

In [ ]:
import momepy as mp
if edges.crs.is_geographic:
    edges.set_crs(crs=32633, allow_override=True, inplace=True)
tess = mp.Tessellation(edges, unique_id = edge_key, limit=aoi)
t = tess.tessellation
print(f"Tessellation created {len(tess.multipolygons)} multipolygons.")

In [ ]:
if compare_net_file is not None:
    if compare_edges.crs.is_geographic:
        compare_edges.set_crs(crs=32633, allow_override=True, inplace=True)
    compare_tess = mp.Tessellation(compare_edges, unique_id = "edge_id", limit=aoi)
    compare_t = compare_tess.tessellation
    print(f"Tessellation created {len(compare_tess.multipolygons)} multipolygons.")

In [ ]:
t['area'] = t.geometry.area
t.area.describe()

In [ ]:
if compare_net_file is not None:
    compare_t['area'] = compare_t.geometry.area
    compare_t.area.describe()

In [ ]:
# re-index t to match edge_id (for joining with edges)
t.set_index(edge_key, inplace=True)

In [ ]:
if nodes.crs.is_geographic:
        nodes.set_crs(crs=32633, allow_override=True, inplace=True)
nodes.explore()

In [ ]:
edges.explore(tooltip=['edge_id', 'from_node', 'to_node'])

In [ ]:
map = net_routing.explore(tiles="CartoDB positron", color="#A6D62A")
map = t.explore(m=map, color="#E2582C", style_kwds={"weight":1, "fillOpacity":0.2})
nodes.explore(m=map, color="#1F9F5E")

In [ ]:
print(f"Plotting tessellation")
ax = t.boundary.plot(edgecolor="red", lw=0.2)
ax = edges.plot(ax=ax, lw=0.5)
ax = nodes.plot(ax=ax, markersize=0.5)
plt.margins(0)
plt.axis('off')
plt.savefig(fname=f"plots/{case_id}_tessel.pdf", bbox_inches='tight')
plt.savefig(fname=f"plots/svg/{case_id}_tessel.svg", bbox_inches='tight')
plt.show()

### accumulate and assign edge-based weights per node

In [ ]:
def compute_node_weights(edges, edge_weights):
    # join network (edge list) f/t node columns with edge weight column
    tmp = edges[['from_node','to_node', 'geometry']].join(edge_weights) # edge-based weight
    display(tmp.explore(column="area"))
    wa = tmp.groupby(['from_node']).sum()[['area']]/2 # split edge weight to end nodes per edge
    wa.index.rename("nid", inplace=True)
    wb = tmp.groupby(['to_node']).sum()['area']/2
    wb.index.rename("nid", inplace=True)
    w = wa.join(wb, lsuffix='_a', rsuffix='_b', how='outer') # begin aggregation for each node role (from/to)
    w.fillna(0, inplace=True) # set zero weight for nodes with non-existing weight input data
    w['weight'] = w.area_a + w.area_b
    dif = w.weight.sum() - edge_weights.sum()[0]
    print("Sum of node weights", "equals" if abs(dif) < 0.0001 else "DOES NOT MATCH", "sum of edge weights")
    print(f">>> total dif: {dif:.4f} --- sum of node weights: {w.weight.sum():.2f}, sum of edge weights: {edge_weights.sum()[0]:.2f}")
    print(f">>> this equals to {dif/edge_weights.sum()[0]:.1%}")
    return w
w = compute_node_weights(edges, t[['area']])
if compare_net_file is not None:
    compare_w = compute_node_weights(compare_edges, compare_t[['area']])
w.head()

In [ ]:
nodes["weight"] = w.weight
nodes.explore(column="weight")

In [ ]:
# for reference only: option to pre-compute the full interaction matrix (e.g. for use with the SIBC approach by Wu et al. 2022)
def compute_interaction(node_weights):
    nw_dict = node_weights[['weight']].to_dict()['weight']
    interaction_dict = {}
    sum_nw = node_weights[['weight']].sum().weight
    node_weights['rel_weight'] = node_weights.weight / sum_nw
    nw_rel_dict = node_weights[['rel_weight']].to_dict()['rel_weight']

    for k1 in nw_dict:
        for k2 in nw_rel_dict:
            # w_orig * share_dest -> distribute w_orig across dest nodes
            interaction_dict.setdefault((k1, k2), nw_dict[k1] * nw_rel_dict[k2])
    
    return interaction_dict 

## compute spatially normalised betweenness centrality

In [ ]:
import algo.centrality as sbc
c_sp_sbc = sbc.spatial_betweenness_centrality(g, w, w, normalized=normalize, weight='length')
nh.add_centr_to_netascore(edges, c_sp_sbc, "sp_sbc", edge_key)

In [ ]:
if compare_net_file is not None:
    compare_c_sp_sbc = sbc.spatial_betweenness_centrality(compare_g, compare_w, compare_w, normalized=normalize, weight='length')
    nh.add_centr_to_netascore(compare_edges, compare_c_sp_sbc, "sp_sbc", edge_key)

## compare and explore results

In [ ]:
edges.explore(column='centr_sp_sum', tiles="CartoDB darkmatter", tooltip=['centr_sp_sum', 'centr_sp_sbc_sum', edge_key])

In [ ]:
edges.explore(column='centr_sp_sbc_sum', tiles="CartoDB darkmatter", tooltip=['osm_id', 'centr_sp_sbc_sum', 'centr_sp_sbc_ft', 'centr_sp_sbc_tf'])

In [ ]:
if compare_net_file is not None:
    display(compare_edges.explore(column='centr_sp_sbc_sum', tiles="CartoDB darkmatter", tooltip=['centr_sp_sbc_sum', 'centr_sp_sbc_ft', 'centr_sp_sbc_tf']))

In [ ]:
if compare_net_file is not None:
    # compute difference between case and compare_case
    d_a = edges[['edge_id', 'osm_id', 'from_node', 'to_node', 'geometry', 'centr_sp_sum', 'centr_sp_sbc_sum']].set_index('osm_id')
    d_b = compare_edges[['edge_id', 'osm_id', 'from_node', 'to_node', 'centr_sp_sum', 'centr_sp_sbc_sum']].set_index('osm_id')
    comp = d_a.join(d_b, lsuffix='_orig', rsuffix='_comp')
    comp['cc_centr_sp_sum_dif'] = comp['centr_sp_sum_orig'] - comp['centr_sp_sum_comp']
    comp['cc_centr_sp_sum_dif_rel'] = comp['cc_centr_sp_sum_dif'] / comp['centr_sp_sum_comp'] * 100
    comp['cc_centr_sp_sbc_sum_dif'] = comp['centr_sp_sbc_sum_orig'] - comp['centr_sp_sbc_sum_comp']
    comp['cc_centr_sp_sbc_sum_dif_rel'] = comp['cc_centr_sp_sbc_sum_dif'] / comp['centr_sp_sbc_sum_comp'] * 100
    
    # also add values for edges in subdivision
    subdiv_ids = comp[[v.startswith('s') for v in comp.index.values]].index.values
    for sid in subdiv_ids:
        parts = sid.split(".")
        if len(parts) < 4:
            # no matching counterpart (inner subdiv)
            # set centr_sp_sum_comp to 0
            comp.loc[comp.index == sid, 'centr_sp_sum_comp'] = 0
            comp.loc[comp.index == sid, 'centr_sp_sbc_sum_comp'] = 0
            # set cc_centr_sp_sum_dif to orig - comp -> orig
            comp.loc[comp.index == sid, 'cc_centr_sp_sum_dif'] = comp.loc[comp.index == sid, 'centr_sp_sum_orig']
            comp.loc[comp.index == sid, 'cc_centr_sp_sbc_sum_dif'] = comp.loc[comp.index == sid, 'centr_sp_sbc_sum_orig']
            # set (leave) cc_centr_sp_sum_dif_rel to NaN -> relative increase is +inf
        else:
            # set centr_sp_sum_comp to value of h/v.id
            comp_val = d_b.loc[d_b.index == f"{parts[2]}.{parts[3]}", 'centr_sp_sum'].values
            comp_val_s = d_b.loc[d_b.index == f"{parts[2]}.{parts[3]}", 'centr_sp_sbc_sum'].values
            comp.loc[comp.index == sid, 'centr_sp_sum_comp'] = comp_val
            comp.loc[comp.index == sid, 'centr_sp_sbc_sum_comp'] = comp_val_s
            # set cc_centr_sp_sum_dif to orig - comp
            dif_val = comp.loc[comp.index == sid, 'centr_sp_sum_orig'] - comp_val
            comp.loc[comp.index == sid, 'cc_centr_sp_sum_dif'] = dif_val
            dif_val_s = comp.loc[comp.index == sid, 'centr_sp_sbc_sum_orig'] - comp_val_s
            comp.loc[comp.index == sid, 'cc_centr_sp_sbc_sum_dif'] = dif_val_s
            # set cc_centr_sp_sum_dif_rel to dif / comp * 100
            comp.loc[comp.index == sid, 'cc_centr_sp_sum_dif_rel'] = dif_val / comp_val * 100
            comp.loc[comp.index == sid, 'cc_centr_sp_sbc_sum_dif_rel'] = dif_val_s / comp_val_s * 100
    comp.head()

In [ ]:
if compare_net_file is not None:
    display(comp.explore(column='cc_centr_sp_sum_dif_rel', tiles="CartoDB darkmatter", tooltip=['centr_sp_sum_orig', 'centr_sp_sum_comp']))

In [ ]:
edges['dif_centr_sp-sbc'] = edges['centr_sp_sbc_sum'] - edges['centr_sp_sum']
edges['rel_dif_centr_sp-sbc'] = (edges['centr_sp_sbc_sum'] - edges['centr_sp_sum']) / edges['centr_sp_sum'] * 100

In [ ]:
comp_col = 'dif_centr_sp-sbc'
maxval = np.abs(edges[comp_col].min())
if np.abs(edges[comp_col].max()) > maxval:
    maxval = np.abs(edges[comp_col].max())
edges[edges[comp_col]!=0].explore(column=comp_col, tiles='CartoDB positron', cmap='RdBu', vmin = 0-maxval, vmax = maxval,
        tooltip=[comp_col, 'centr_sp_sum', 'centr_sp_sbc_sum'])

In [ ]:
comp_col = 'rel_dif_centr_sp-sbc'
maxval = np.abs(edges[comp_col].min())
if np.abs(edges[comp_col].max()) > maxval:
    maxval = np.abs(edges[comp_col].max())
edges[edges[comp_col]!=0].explore(column=comp_col, tiles='CartoDB positron', cmap='RdBu', vmin = 0-maxval, vmax = maxval,
        tooltip=[comp_col, 'centr_sp_sum',  'centr_sp_sbc_sum'])

In [ ]:
def centr_plot(data, centr_col, label):
    print(f"Plotting column '{centr_col}'")
    #bins = compute_bins(data[delta_col], fac)
    data.plot(column=centr_col, cmap=cmap_centr, legend=True, scheme="equalinterval", 
            #x_ticks=None,
            classification_kwds={'k':7}, 
            legend_kwds={"fmt": "{:.3f}", "interval":True,
                            "loc":'upper left',
                            'bbox_to_anchor':(1.01,1),
                            'frameon':False,
                            'title':f'${label}$\n',
                            'alignment':'center',
                            'handletextpad':0.1,
                            'labelspacing':0.65,
                            'title_fontproperties':{'size':'large'}}
            ) #'lowest': -d_max # known bug: https://github.com/pysal/mapclassify/issues/175
    plt.margins(0)
    plt.axis('off')
    plt.savefig(fname=f"plots/{case_id}_{centr_col}.pdf", bbox_inches='tight')
    plt.savefig(fname=f"plots/svg/{case_id}_{centr_col}.svg", bbox_inches='tight')
    plt.show()

In [ ]:
def compute_bins(data, fac=1):
    d_max = abs(data.min()*fac)
    if data.max()*fac > d_max:
        d_max = data.max()*fac
    d_max = math.ceil(d_max)
    tmp = d_max
    k = 7
    d_max = math.ceil(d_max/k)*k
    dif = d_max*2.0/k
    cur = -d_max
    print(f"max val: {tmp}, dif: {dif}")
    bins = [] # upper bin bounds are used - the lowest needs to be skipped - otherwise "-inf" will be used as lower bound
    while d_max - cur - dif > 0.001:
        cur += dif
        bins.append(cur)
    bins.append(tmp)
    bins = [x / fac for x in bins]
    print(bins)
    return bins

def delta_plot(data, delta_col, label, unit = " [%]", fac=1):
    print(f"Plotting column '{delta_col}'")
    bins = compute_bins(data[delta_col], fac)
    data.plot(column=delta_col, cmap=cmap_diverging, legend=True, scheme="UserDefined", 
            classification_kwds={'bins':bins}, 
            legend_kwds={"fmt": "{:." + str(len(str(fac))-1) + "f}", "interval":True,
                            "loc":'upper left',
                            'bbox_to_anchor':(1.01,1),
                            'frameon':False,
                            'title':f'$\Delta {label}${unit}\n',
                            'alignment':'center',
                            'handletextpad':0.1,
                            'labelspacing':0.65,
                            'title_fontproperties':{'size':'large'}}
            ) #'lowest': -d_max # known bug: https://github.com/pysal/mapclassify/issues/175
    plt.margins(0)
    plt.axis('off')
    plt.savefig(fname=f"plots/{case_id}_{delta_col}.pdf", bbox_inches='tight')
    plt.savefig(fname=f"plots/svg/{case_id}_{delta_col}.svg", bbox_inches='tight')
    plt.show()

In [ ]:
if compare_net_file is not None:
    centr_plot(comp, 'centr_sp_sum_comp', 'c_Bref')
    centr_plot(comp, 'centr_sp_sbc_sum_comp', 'c_{SB}ref')
centr_plot(edges, 'centr_sp_sum', 'c_B')
centr_plot(edges, 'centr_sp_sbc_sum', 'c_{SB}')

In [ ]:
delta_plot(edges, 'rel_dif_centr_sp-sbc', 'c_B,c_{SB}')
delta_plot(edges, 'dif_centr_sp-sbc', 'c_B,c_{SB}', unit="", fac=100000)
if compare_net_file is not None:
    delta_plot(comp, 'cc_centr_sp_sum_dif_rel', 'c_B{ref},c_B')
    delta_plot(comp, 'cc_centr_sp_sum_dif', 'c_B{ref},c_B', unit="", fac=10000)
    delta_plot(comp, 'cc_centr_sp_sbc_sum_dif_rel', 'c_{SB}{ref},c_{SB}')
    delta_plot(comp, 'cc_centr_sp_sbc_sum_dif', 'c_{SB}{ref},c_{SB}', unit="", fac=10000)

In [ ]:
if compare_net_file is not None:
    comp[comp.cc_centr_sp_sbc_sum_dif_rel > 10]

In [ ]:
edges.to_file(f"net_out/{case_id}.gpkg")